In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [2]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

2017-06-17 22:48:30,964 [INFO] [holmes.experiment.staging] Preparing holmes payload
2017-06-17 22:48:30,966 [INFO] [holmes.experiment.staging] Cloning git:hudl/holmes
2017-06-17 22:48:33,878 [INFO] [holmes.experiment.staging] Checking out git:hudl/holmes:ca6ccd63f270d603ffa9bdf5a7ced184d6339796
2017-06-17 22:48:34,221 [INFO] [holmes.experiment.staging] Preparing distribution package
2017-06-17 22:48:36,927 [INFO] [holmes.experiment.staging] Staging distribution package
2017-06-17 22:48:36,928 [INFO] [holmes.experiment.staging] Commit staged
2017-06-17 22:48:36,929 [INFO] [holmes.experiment.staging] Removing Holmes Directory
2017-06-17 22:48:36,982 [INFO] [holmes.experiment.staging] Extracting default config from package


In [3]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_thresholds = [4,5]
hungarian_thresholds = [10,15,20]
minimum_detections = [25]

In [4]:
list_configs=[]
for lt in lost_thresholds:
    for ht in hungarian_thresholds:
        for min_det in minimum_detections:
            for config in list_dicts:
                current_config_dict = config.copy()
                parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt,"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
                flat_dict = flatten_dict(current_config_dict)
                flat_dict.update(parameter_map)
                updated_config_dict = expand_dict(flat_dict)
                updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
                file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}-{:02d}-{:02d}.json".format(len(list_configs), 
                                                    current_config_dict['video_config']['video_id'], lt, ht, min_det))
                list_configs.append(file_name)
                with open(file_name, 'w') as f:
                    json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

24 /var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpcofbzlhs/M00-ChvUtd-000-04-10-25.json


In [6]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-HT-DET-1-6", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-HT-DET-1-6",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

2017-06-17 22:49:39,532 [INFO] [holmes.experiment.base] Verifying AWS staging area: s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6


In [7]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [8]:
experiment.stage_experiment()

2017-06-17 22:49:42,223 [INFO] [holmes.experiment.base] staging measurement M00-ChvUtd-000-04-10-25
2017-06-17 22:49:42,707 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpcofbzlhs/M00-ChvUtd-000-04-10-25.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/M00-ChvUtd-000-04-10-25.json
2017-06-17 22:49:43,125 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpcofbzlhs/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz
2017-06-17 22:49:44,393 [INFO] [holmes.experiment.base] staging measurement M01-PS0304-001-04-10-25
2017-06-17 22:49:44,913 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpcofbzlhs/M01-PS0304-001-04-10-25.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps

2017-06-17 22:50:05,728 [INFO] [holmes.experiment.base] staging measurement M22-PP0214-000-05-20-25
2017-06-17 22:50:06,197 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpcofbzlhs/M22-PP0214-000-05-20-25.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/M22-PP0214-000-05-20-25.json
2017-06-17 22:50:06,618 [INFO] [holmes.experiment.base] staging measurement M23-ER0429-001-05-20-25
2017-06-17 22:50:07,119 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpcofbzlhs/M23-ER0429-001-05-20-25.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/M23-ER0429-001-05-20-25.json


{'experiment_id': 'PTA-95-LT-HT-DET-1-6',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/M00-ChvUtd-000-04-10-25.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-6',
   'job_id': 'PTA-95-LT-HT-DET-1-6-M00-ChvUtd-000-04-10-25',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M00-ChvUtd-000-04-10-25',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz'},
  {'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/M01-PS0304-001-04-10-25.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-6',
   'job_id': 'PTA-95-LT-HT-DET-1-6-M01-PS0304-001-04-10-25',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M01-PS0304-001-04-10-25',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/ca6ccd63f270d603ffa9bdf5a7ced184d63397

In [9]:
experiment.launch_experiment()

2017-06-17 22:50:09,276 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M00-ChvUtd-000-04-10-25
2017-06-17 22:50:09,968 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M01-PS0304-001-04-10-25
2017-06-17 22:50:10,238 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M02-PP0214-000-04-10-25
2017-06-17 22:50:10,510 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M03-ER0429-001-04-10-25
2017-06-17 22:50:10,781 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M04-ChvUtd-000-04-15-25
2017-06-17 22:50:11,040 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M05-PS0304-001-04-15-25
2017-06-17 22:50:11,336 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M06-PP0214-000-04-15-25
2017-06-17 22:50:11,610 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M07-ER0429-001-04-15-25
2017-06-17 22:50:11,894 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-6-M08-ChvUtd-000-04-20-25
2

{'experiment_id': 'PTA-95-LT-HT-DET-1-6',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-6/deps/M00-ChvUtd-000-04-10-25.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-6',
   'job_id': 'PTA-95-LT-HT-DET-1-6-M00-ChvUtd-000-04-10-25',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
      'content-length': '105',
      'content-type': 'application/json',
      'date': 'Sun, 18 Jun 2017 03:47:56 GMT',
      'via': '1.1 0a9f4502819b08c3a7919c963887be2b.cloudfront.net (CloudFront)',
      'x-amz-cf-id': 'X7i8L-bl0G6u4iAD9N4YRAh36VVLfDRYM4rZBE5AN7DYlct9mSnX-w==',
      'x-amzn-requestid': 'ea2e3f11-53d8-11e7-bf53-f5465514d7e7',
      'x-amzn-trace-id': 'sampled=0;root=1-5945f7ec-859496a7b42338c29911db58',
      'x-cache': 'Miss from cloudfront'},
     'HTTPStatusCode': 200,
     'RequestId': 'ea2e3f11-53d8-11e7-bf53-f5465514d7e7',


In [10]:
import pandas as pd

In [11]:
data = pd.DataFrame({'mota':[.672,.672,.669,.670,.670,.667,.858,.865,.865,.859,.864,.864,.769,.794,.796,.769,.794,
                             .795,.754,.767,.771,.756,.769,.771],
              'lt':[4,4,4,5,5,5,4,4,4,5,5,5,4,4,4,5,5,5,4,4,4,5,5,5],
              'ht':[10,15,20,10,15,20,10,15,20,10,15,20,10,15,20,10,15,20,10,15,20,10,15,20],
              'det':[25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25],
             'game':['ER0429','ER0429','ER0429','ER0429','ER0429','ER0429','PP0214','PP0214','PP0214','PP0214',
                     'PP0214','PP0214','ChvUtd','ChvUtd','ChvUtd','ChvUtd','ChvUtd','ChvUtd','PS0304','PS0304',
                     'PS0304','PS0304','PS0304','PS0304']})

In [13]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [14]:
output_notebook()

Loading BokehJS ...

In [15]:
data_lt_ht_det = data.copy()
data_lt_ht_det['lost'] = data_lt_ht_det['lt']
data_lt_ht_det['hungarian'] = data_lt_ht_det['ht']
data_lt_ht_det['detections'] = data_lt_ht_det['det']
data_lt_ht_det = data_lt_ht_det.loc[:,['mota','ht', 'lt', 'det', 'lost', 'hungarian', 'detections']].groupby(['ht','lt','det']).mean()
data_lt_ht_det

mota  lost  hungarian  detections
ht lt det                                      
10 4  25   0.76325     4         10          25
   5  25   0.76350     5         10          25
15 4  25   0.77450     4         15          25
   5  25   0.77425     5         15          25
20 4  25   0.77525     4         20          25
   5  25   0.77425     5         20          25

In [17]:
scat_lt_ht_det_1 = Scatter(data_lt_ht_det, x='lost', y='mota', color='hungarian', marker='detections')
show(scat_lt_ht_det_1)